# Imports

In [1]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

# Dash framework and core components
import dash
from dash import dcc, html, ctx
from dash.dependencies import Input, Output
import dash_player

# Dash Bootstrap Components for styling
import dash_bootstrap_components as dbc

# Plotly for interactive plotting
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from ipywidgets import Button, Checkbox, GridBox, Layout, Label, FloatSlider, IntText, Text, FloatText

# Pandas for data manipulation
import pandas as pd
import numpy as np
import pickle
import csv
import json

# Additional Python libraries
import time
from datetime import timedelta

# system libraries
import sys
import os
import shutil
import glob

# second interface
from prediction import split, model
from sklearn.model_selection import train_test_split

# third interface
import math
from tensorflow.keras.models import load_model
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns

# List all files in the directory
files = glob.glob('assets/' + '*')

# Delete each file
for file in files:
    os.remove(file)

# Fetch Data

In [2]:
info = widgets.HTML(
    value='<h2 style="font-weight: bold; color: black;">Fetch Data</h2>',
    placeholder='',
    description='',
)
display(info)

# Add a text widget for specifying the path to the raw TS data
raw_ts_path_input = widgets.Text(
    value='',
    placeholder='Enter path to your data here',
    description='Data Path:',
    disabled=False
)
display(raw_ts_path_input)

# Function to load the raw TS data and copy it to the assets directory
def load_and_copy_raw_ts(path):
    assets_dir = "assets"
    output_file_name = "manual_label_df.csv"  # Desired file name

    if os.path.isfile(path):
        if not os.path.isdir(assets_dir):
            os.makedirs(assets_dir)

        shutil.copy(path, os.path.join(assets_dir, output_file_name))
        print(f"Data to manually label copied to {os.path.join(assets_dir, output_file_name)}.")
    else:
        print("Data file not found.")

# Button to load the raw TS data path
load_raw_ts_button = widgets.Button(description="Load Data")

# Callback for the load raw TS data path button
def on_load_raw_ts_button_clicked(b):
    global raw_ts_path
    raw_ts_path = load_and_copy_raw_ts(raw_ts_path_input.value)

load_raw_ts_button.on_click(on_load_raw_ts_button_clicked)
display(load_raw_ts_button)

# Add a text widget for specifying the path to the list of labels
labels_path_input = widgets.Text(
    value='',
    placeholder='Enter path to your label list here',
    description='Label Path:',
    disabled=False
)
display(labels_path_input)

# Function to load the labels list and copy it to the assets directory
def load_and_copy_labels(path):
    assets_dir = "assets"
    output_file_name = "label_list.csv"  # Desired file name

    if os.path.isfile(path):
        if not os.path.isdir(assets_dir):
            os.makedirs(assets_dir)

        shutil.copy(path, os.path.join(assets_dir, output_file_name))
        print(f"Label list copied to {os.path.join(assets_dir, output_file_name)}.")
    else:
        print("Label list file not found.")

# Button to load the labels list path
load_labels_button = widgets.Button(description="Load Label List")

# Callback for the load labels list path button
def on_load_labels_button_clicked(b):
    global labels_path
    labels_path = load_and_copy_labels(labels_path_input.value)

load_labels_button.on_click(on_load_labels_button_clicked)
display(load_labels_button)

# Add a text widget for specifying the video path
video_path_input = widgets.Text(
    value='',
    placeholder='Enter path to your video here',
    description='Video Path:',
    disabled=False
)
display(video_path_input)

# Function to load the video path and copy the video to the assets directory
def load_and_copy_video(path):
    assets_dir = "assets"
    output_file_name = "manual_video.mp4"  # Desired file name

    if os.path.isfile(path):
        if not os.path.isdir(assets_dir):
            os.makedirs(assets_dir)

        shutil.copy(path, os.path.join(assets_dir, output_file_name))
        print(f"Video copied to {os.path.join(assets_dir, output_file_name)}.")
    else:
        print("Video file not found.")

# Button to load the video path
load_video_button = widgets.Button(description="Load Video Path")

# Callback for the load video path button
def on_load_video_button_clicked(b):
    global video_path
    video_path = load_and_copy_video(video_path_input.value)

load_video_button.on_click(on_load_video_button_clicked)
display(load_video_button)

HTML(value='<h2 style="font-weight: bold; color: black;">Fetch Data</h2>', placeholder='')

Text(value='', description='Data Path:', placeholder='Enter path to your data here')

Button(description='Load Data', style=ButtonStyle())

Text(value='', description='Label Path:', placeholder='Enter path to your label list here')

Button(description='Load Label List', style=ButtonStyle())

Text(value='', description='Video Path:', placeholder='Enter path to your video here')

Button(description='Load Video Path', style=ButtonStyle())

Data to manually label copied to assets\manual_label_df.csv.
Label list copied to assets\label_list.csv.
Video copied to assets\manual_video.mp4.


In [3]:
df = pd.read_csv('assets/manual_label_df.csv') # time-series data
labelListDF = pd.read_csv('assets/label_list.csv') # list of labels for ts data
video_path = 'assets/manual_video.mp4' # local video in asset directory
cols = list(df.columns)

def create_feature_selection_ui(df, title, output_json, key_selected, key_omitted=None):
    info = Label(title)
    display(info)

    cols = list(df.columns)
    checkboxes = [Checkbox(value=False, description=label) for label in cols]
    grid = GridBox(children=checkboxes, layout=Layout(grid_template_columns="repeat(4, 1fr)"))
    display(grid)
    
    submit_button = Button(description="Submit")
    display(submit_button)

    def handle_submit(sender):
        selected_data = [cb.description for cb in checkboxes if cb.value]
        omitted_data = [cb.description for cb in checkboxes if not cb.value]

        # Save to JSON file
        data_to_save = {}
        try:
            with open(output_json, 'r') as file:
                data_to_save = json.load(file)
        except FileNotFoundError:
            pass  # File doesn't exist, we'll create it

        data_to_save[key_selected] = selected_data
        if key_omitted:
            data_to_save[key_omitted] = omitted_data

        with open(output_json, 'w') as file:
            json.dump(data_to_save, file, indent=4)

        print(f"{key_selected}: ", selected_data)
        if key_omitted:
            print(f"{key_omitted}: ", omitted_data)
  
    submit_button.on_click(handle_submit)

df = pd.read_csv('assets/manual_label_df.csv')

# Stage 1: Selecting valid features
create_feature_selection_ui(df, 
                            'Only select the features you consider valid\n', 
                            'assets/config.json', 
                            'valid_features', 
                            'features_to_omit')

# Stage 2: Selecting features to show initially
create_feature_selection_ui(df, 
                            'Select features to show on the first render of the graph\n', 
                            'assets/config.json', 
                            'features_to_show')

def create_number_input_ui(title, output_json, key, min_val=None, max_val=None, step=None, default=0):
    info = Label(value=title)
    display(info)
    
    if min_val is not None and max_val is not None and step is not None:
        # Use a slider for input
        number_input = FloatSlider(value=default, min=min_val, max=max_val, step=step, description=key)
    else:
        # Use an integer text box for input
        number_input = IntText(value=default, description=key)
    display(number_input)

    submit_button = Button(description="Submit")
    display(submit_button)

    def handle_submit(sender):
        input_value = number_input.value

        # Save to JSON file
        data_to_save = {}
        try:
            with open(output_json, 'r') as file:
                data_to_save = json.load(file)
        except FileNotFoundError:
            pass  # File doesn't exist, we'll create it

        data_to_save[key] = input_value

        with open(output_json, 'w') as file:
            json.dump(data_to_save, file, indent=4)

        print(f"{key}: {input_value}")

    submit_button.on_click(handle_submit)

# Stage 3: Input for Confidence Threshold
create_number_input_ui('Enter the confidence threshold (0.0 - 1.0):\n',
                       'assets/config.json', 
                       'conf_thresh',
                       min_val=0.0, max_val=1.0, step=0.01, default=0.5)

# Stage 4: Input for Window Size
create_number_input_ui('Enter the window size value (integer):\n',
                       'assets/config.json', 
                       'window_size')

# Stage 5: Input for Offset
create_number_input_ui('Enter the offset value in seconds (integer) (MUST ME MULTIPLE OF WINDOW SIZE *WORD BETTER LATER*... 96 (window) samples = 3s in TWristAR):\n',
                       'assets/config.json', 
                       'offset')


Label(value='Only select the features you consider valid\n')

GridBox(children=(Checkbox(value=False, description='datetime'), Checkbox(value=False, description='accel_x'),…

Button(description='Submit', style=ButtonStyle())

Label(value='Select features to show on the first render of the graph\n')

GridBox(children=(Checkbox(value=False, description='datetime'), Checkbox(value=False, description='accel_x'),…

...

Button(description='Submit', style=ButtonStyle())

Label(value='Enter the offset value in seconds (integer) (MUST ME MULTIPLE OF WINDOW SIZE *WORD BETTER LATER*.…

IntText(value=0, description='offset')

Button(description='Submit', style=ButtonStyle())

offset: 0
window_size: 95
conf_thresh: 0.9
features_to_show:  ['accel_ttl']
valid_features:  ['accel_x', 'accel_y', 'accel_z', 'accel_ttl']
features_to_omit:  ['datetime', 'label', 'sub']


## Colors

In [4]:
colorList = ('#4363d8', '#e6194b', '#3cb44b', '#f58231', '#911eb4', '#46f0f0', '#f032e6', '#bcf60c', '#fabebe',
             '#008080', '#e6beff', '#9a6324', '#800000', '#aaffc3', '#808000', '#ffd8b1', '#000075', '#808080',
             '#000000')
# This assigns a specific color from colorList to each label in provided label list ** IF LABEL IS UNDEFINED COLOR IS BLACK**
#colorDict = {label: ('#000000' if label == 'Undefined' else color) for label, color in zip(labelListDF, colorList)}

# Modify the dictionary comprehension to cycle through the color list (> 20 labels)
colorDict = {label: (colorList[i % len(colorList)] if label != 'Undefined' else '#000000') 
             for i, label in enumerate(labelListDF)}

# Confidence list & dictionary
confidenceValues = ("High", "Medium", "Low", "Undefined")
confidenceColors = ('#3cb44b', '#ffe119', '#FF0000', '#000000')

confDict = dict(zip(confidenceValues, confidenceColors))

# Load selected features from the JSON configuration
with open('assets/config.json', 'r') as file:
    config = json.load(file)
cols = config.get('valid_features', [])

# if df has a confidence column, remove it from column list
if "confidence" in cols:
    pass
    # cols.remove('confidence')
# else, make a confidence column assigned an undefined
else:
    df['confidence'] = "Undefined"

# if df has a label column, remove it from column list
if "label" in df.columns:
    pass
    # cols.remove('label')
# else, make a label column assigned an undefined
else:
    df['label'] = "Undefined"

# Should be an interface to provide paths later.

df.to_csv('assets/manual_label_df.csv', index=False)
with open('assets/feature_cols.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(cols)

# First Interface

In [5]:
%run 1st_app.py

Dash app running on http://127.0.0.1:8008/


# Build Model

In [6]:
info = widgets.Label('Specifications for length of timesteps: i.e. (highest_frequency * seconds)')
display(info)
window_size= widgets.IntText(
    value=96,
    description='Window Size:',
    disabled=False
)
freq = widgets.Output()
display(window_size, freq)


steps = widgets.IntText(
    value= 96,
    description='Steps:',
    disabled=False
)
step = widgets.Output()
display(steps, step)

Label(value='Specifications for length of timesteps: i.e. (highest_frequency * seconds)')

IntText(value=96, description='Window Size:')

Output()

IntText(value=96, description='Steps:')

Output()

In [7]:
# NECESSARY STUFF TO RELOAD AFTER CHANGES IN 1ST DASH APP
df = pd.read_csv('assets/manual_label_df.csv') # time-series data
labelListDF = pd.read_csv('assets/label_list.csv') # list of labels for ts data
labelList = list(labelListDF)

# Build Model Begin
cnn = model.CNN()
newDf = df.copy()
newDf = newDf.drop('confidence',axis = 1)
newDf = newDf.set_index('datetime')

print("Available Labels: ", labelList)
print("Encoding labels present in the data")

# this is taken from the UI, how to get it here?
new_np = split.TimeSeriesNP(window_size.value, steps.value)

new_np.setArrays(newDf, encode=True, one_hot_encode=False ,labels= labelList)
print(new_np.mapping)

print("x shape is: "+ str(new_np.x.shape))
print("y shape is: " + str(new_np.y.shape))

x_train, x_validation, y_train, y_validation = train_test_split(new_np.x, new_np.y.ravel(), test_size = 0.25)
cnn.only_train_data(x_train, x_validation, y_train, y_validation )

info = widgets.Label('Name your model if you want to save it')
display(info)
modelName = widgets.Text(
    value = "model",
    placeholder='1D_CNN',
    description='Model Name:',
    disabled=False
)
q = widgets.Output()
display(modelName,q)

# save new_np to assets folder as pickle
with open(os.path.join('assets', "time_series_processed_data.pkl"), 'wb') as file:
    pickle.dump(new_np, file)

#credit: https://stackoverflow.com/questions/58716248/how-to-create-a-simple-button-with-output-in-a-python-jupyter-notebook
info = widgets.Label('This button will construct and train the model')
display(info)

def clicked(arg):
    print("Please wait while model is built.")
    cnn.build()
    cnn.train()
    cnn.model.summary()

button_download = widgets.Button(description = 'Build')   
button_download.on_click(clicked)
display(button_download)

Available Labels:  ['Downstairs', 'Jogging', 'Sitting', 'Standing', 'Upstairs', 'Walking', 'Other']
Encoding labels present in the data
{0: 'Downstairs', 1: 'Jogging', 2: 'Sitting', 3: 'Standing', 4: 'Upstairs', 5: 'Walking', 6: 'Other'}
x shape is: (123, 96, 4)
y shape is: (123,)


Label(value='Name your model if you want to save it')

Text(value='model', description='Model Name:', placeholder='1D_CNN')

Output()

Label(value='This button will construct and train the model')

Button(description='Build', style=ButtonStyle())

# Save Model

In [8]:
from typing_extensions import ClassVar
import csv

original_cwd = os.getcwd()

def clicked(arg):
    try:
        # Change to the models directory
        models_dir = os.path.join('prediction', 'models')
        os.makedirs(models_dir, exist_ok=True)  # Create directory if it doesn't exist
        os.chdir(models_dir)
        cnn.model.save(modelName.value + ".h5")

        # Change to the dimensions directory
        dimensions_dir = os.path.join('..', 'dimensions')
        os.makedirs(dimensions_dir, exist_ok=True)  # Create directory if it doesn't exist
        os.chdir(dimensions_dir)
        with open(modelName.value + ".csv", 'w', newline='') as f:
            print("Please wait while the model is being saved.")
            writer = csv.writer(f)
            writer.writerow([window_size.value, steps.value])
            print("Model Saved.")
    finally:
        # Reset to the original working directory
        os.chdir(original_cwd)

info = widgets.Label('This button will save the model')
display(info)
button_download = widgets.Button(description='Save Model')   
button_download.on_click(clicked)
display(button_download)

Label(value='This button will save the model')

Button(description='Save Model', style=ButtonStyle())

# Load Model &
# Upload Dataset for Auto-Labeling

In [9]:
from IPython.display import HTML

radiobut = True
model_select = widgets.RadioButtons(
        options=[''],
        value= '',
        description='Models:',
        disabled=False
    )

# Custom CSS to change radio button label color to black
style = """
<style>
    .widget-radio label { /* Target labels in radio button widget */
        color: black !important; /* Set font color to black */
    }
</style>
"""
display(HTML(style))

if os.path.isdir(os.path.join("prediction","models")):
    dir_list = os.listdir(os.path.join("prediction", "models"))
    model_select.options = dir_list

else:
    radiobut = False
    model_select = widgets.Text(
    value='',
    placeholder='Type something',
    description='File Path:',
    disabled=False
)
# Button for submitting the model selection
submit_button = widgets.Button(description="Submit Selection")

# Callback function for the button
def on_submit_button_clicked(b):
    selected_model_path = os.path.join("prediction", "models", model_select.value) if radiobut else model_select.value
    assets_dir = "assets"  # Define the assets directory path

    # Check if the selected model exists and the assets directory exists, then copy
    if os.path.isfile(selected_model_path) and os.path.isdir(assets_dir):
        shutil.copy(selected_model_path, assets_dir)
        print("Model copied to assets directory.")
    else:
        print("Model not found or assets directory does not exist.")

# Bind the callback function to the button
submit_button.on_click(on_submit_button_clicked)

# Display the UI elements
info = widgets.Label('Select or Upload Model')
display(info)
display(model_select)
display(submit_button)

os.chdir(os.path.join('data'))
info = widgets.Label('Upload dataset you wish to have labeled')
# Colab Upload
# uploaded = files.upload()
# file_c = list(uploaded.keys())
# to_label_df = pd.read_csv(io.StringIO(uploaded[file_c[len(file_c) - 1]].decode('utf-8')), sep = ',')

# New code for dataset path input
data_path_input = widgets.Text(
    value='',
    placeholder='Enter path to your dataset here',
    description='Data Path:',
    disabled=False
)
display(info)
display(data_path_input)

# Function to load and save the dataset
def load_and_save_dataset(path):
    try:
        temp_df = pd.read_csv(path, sep=',')
        print("Dataset loaded successfully.")

        assets_dir = "assets"  # Define the assets directory path
        save_path = os.path.join(assets_dir, "auto_label_df.csv") # Fixed file name

        temp_df.to_csv(save_path, index=False)
        print(f"Dataset saved to {save_path}")

        return temp_df
    except Exception as e:
        print(f"Error loading dataset: {e}")
        return None

# Button to load the dataset
load_data_button = widgets.Button(description="Load Dataset")

# Callback for the load dataset button
def on_load_data_button_clicked(b):
    global to_label_df
    to_label_df = load_and_save_dataset(data_path_input.value)

load_data_button.on_click(on_load_data_button_clicked)
display(load_data_button)

# Add a text widget for specifying the video path
video_path_input = widgets.Text(
    value='',
    placeholder='Enter path to your video here',
    description='Video Path:',
    disabled=False
)
display(video_path_input)

# Function to load the video path and copy the video to the assets directory
def load_and_copy_video(path):
    assets_dir = "assets"
    output_file_name = "autolabel_video.mp4"  # Desired file name

    if os.path.isfile(path):
        if not os.path.isdir(assets_dir):
            os.makedirs(assets_dir)

        shutil.copy(path, os.path.join(assets_dir, output_file_name))
        print(f"Video copied to {os.path.join(assets_dir, output_file_name)}.")
    else:
        print("Video file not found.")

# Button to load the video path
load_video_button = widgets.Button(description="Load Video Path")

# Callback for the load video path button
def on_load_video_button_clicked(b):
    global video_path
    video_path = load_and_copy_video(video_path_input.value)

load_video_button.on_click(on_load_video_button_clicked)
display(load_video_button)

os.chdir(original_cwd)

Label(value='Select or Upload Model')

RadioButtons(description='Models:', options=('TWristARmodel.h5',), value='TWristARmodel.h5')

Button(description='Submit Selection', style=ButtonStyle())

Label(value='Upload dataset you wish to have labeled')

Text(value='', description='Data Path:', placeholder='Enter path to your dataset here')

Button(description='Load Dataset', style=ButtonStyle())

Text(value='', description='Video Path:', placeholder='Enter path to your video here')

Button(description='Load Video Path', style=ButtonStyle())

Model copied to assets directory.
Dataset loaded successfully.
Dataset saved to assets\auto_label_df.csv
Video copied to assets\autolabel_video.mp4.


# Second Dash App (Correct Training Data for Built Model with UMAP)

In [10]:
%run 2nd_app.py



Model loaded from: g:\My Drive\School\time-series-label-assist\assets\TWristARmodel.h5
{'Downstairs': 0, 'Jogging': 1, 'Sitting': 2, 'Standing': 3, 'Upstairs': 4, 'Walking': 5, 'Other': 6}


G:\My Drive\School\time-series-label-assist\2nd_app.py:181: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Standing' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.

c:\Users\trist\miniconda3\envs\dev-alvi\Lib\site-packages\plotly\express\_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



Dash app running on http://127.0.0.1:8080/


c:\Users\trist\miniconda3\envs\dev-alvi\Lib\site-packages\plotly\express\_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.

c:\Users\trist\miniconda3\envs\dev-alvi\Lib\site-packages\plotly\express\_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.

c:\Users\trist\miniconda3\envs\dev-alvi\Lib\site-packages\plotly\express\_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.

c:\Users\trist\miniconda3\envs\dev-alvi\Lib\site-packages\plotly\express\_core.py:2065: FutureWarning:

When grouping with a length-1 list-

UMAP point clicked: {'points': [{'curveNumber': 1, 'pointNumber': 40, 'pointIndex': 40, 'x': -4.401796817779541, 'y': 11.835975646972656, 'hovertext': 'Sitting', 'bbox': {'x0': 249.17000000000002, 'x1': 255.17000000000002, 'y0': 355.84000000000003, 'y1': 361.84000000000003}, 'customdata': [103]}]}
Selected index: 103, Current Label: Standing


c:\Users\trist\miniconda3\envs\dev-alvi\Lib\site-packages\plotly\express\_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.

G:\My Drive\School\time-series-label-assist\2nd_app.py:181: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Standing' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.

c:\Users\trist\miniconda3\envs\dev-alvi\Lib\site-packages\plotly\express\_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.

c:\Users\trist\miniconda3\envs\dev-alvi\Lib\site-packages\plotly\express\_core.py:2065: FutureWarning:

When grouping with a length-1 li

UMAP point clicked: {'points': [{'curveNumber': 1, 'pointNumber': 19, 'pointIndex': 19, 'x': 7.893330097198486, 'y': 3.0846784114837646, 'hovertext': 'Sitting', 'bbox': {'x0': 400.64, 'x1': 406.64, 'y0': 581.42, 'y1': 587.42}, 'customdata': [62]}]}
Selected index: 62, Current Label: Standing


c:\Users\trist\miniconda3\envs\dev-alvi\Lib\site-packages\plotly\express\_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.

c:\Users\trist\miniconda3\envs\dev-alvi\Lib\site-packages\plotly\express\_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.

c:\Users\trist\miniconda3\envs\dev-alvi\Lib\site-packages\plotly\express\_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.

c:\Users\trist\miniconda3\envs\dev-alvi\Lib\site-packages\plotly\express\_core.py:2065: FutureWarning:

When grouping with a length-1 list-

UMAP point clicked: {'points': [{'curveNumber': 0, 'pointNumber': 64, 'pointIndex': 64, 'x': 0.9578282237052917, 'y': 7.916509628295898, 'hovertext': 'Standing', 'bbox': {'x0': 342.32, 'x1': 348.32, 'y0': 478.78, 'y1': 484.78}, 'customdata': [103]}]}
Selected index: 103, Current Label: Standing


c:\Users\trist\miniconda3\envs\dev-alvi\Lib\site-packages\plotly\express\_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.

c:\Users\trist\miniconda3\envs\dev-alvi\Lib\site-packages\plotly\express\_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



## Retrain model + Save

# Predict Labels

#### Specifications for Length of Timesteps

In [ ]:
info = widgets.Label('Specifications for length of timesteps: i.e. (highest_frequency * seconds)')
display(info)
window_size_pred= widgets.IntText(
    value=96,
    description='Window Size:',
    disabled=False
)
freq_pred = widgets.Output()
display(window_size_pred, freq_pred)

steps_pred = widgets.IntText(
    value= 96,
    description='Steps:',
    disabled=False
)
step_pred = widgets.Output()
display(steps_pred, step_pred)

In [ ]:
"""
citations:
getting name of a file without the extension:       https://theprogrammingexpert.com/python-remove-extension/
interesting techniques for labeling df:             https://www.geeksforgeeks.org/label-based-indexing-to-the-pandas-dataframe/
fill in values of df according to dictionary:       https://stackoverflow.com/questions/64481108/assign-a-value-between-index-values-ranges-in-pandas

"""

working_dir = original_cwd

#this one works
def getNPdimensions(modelname, fileloc):
    filename = modelname[:len(modelname) - 3]
    filename = filename + ".csv"
    path = os.path.join(fileloc, filename)

    np_dimensions = []
    if os.path.isfile(path):
        fileList = pd.read_csv(path)
        np_dimensions = list(fileList)
    np_dimensions[0] = int(math.floor(float(np_dimensions[0])))
    np_dimensions[1] = int(math.floor(float(np_dimensions[1])))

    return np_dimensions
###################################################################



if radiobut:
    previousPath = os.getcwd()
    modelPath = os.path.join(working_dir,"prediction","models", model_select.value)
    new_model = load_model(modelPath)
    os.chdir(previousPath)

elif(os.path.isfile(model_select.value)):
    new_model = load_model(model_select.value)

dfA = to_label_df # to_label_df is the df of the uploaded unlabeled set to autolabel
                  # labels_dict = npObject.mapping

# SPECIFIC TO DR. NGU'S DATASET
# dfA = dfA.rename(columns={'p_accelerometer_time' : 'datetime'})
dfA['sub'] = "Undefined"
dfA = dfA.set_index('datetime')
###################################################################

np_dimensions = getNPdimensions(model_select.value , os.path.join(working_dir,"prediction","dimensions"))
#np_labeling = split.TimeSeriesNP(96,32)
np_labeling = split.TimeSeriesNP(window_size_pred.value, steps_pred.value)
print("Label list used:\n", labelList)
np_labeling.setArrays(dfA, encode=True, one_hot_encode=True, labels=labelList, filter=False)

cnn = model.CNN()
testModel = model.CNN()
testModel.setModel(new_model)
testModel.only_test_data(np_labeling.x, np_labeling.y)


####################################################################
def label_confidence(l):
    return max(l)/sum(l)

def pred_to_labels(predictions):
    y_pred = np.argmax(predictions, axis=-1)
    y_pred_labels = np.vectorize(labels_dict.get)(y_pred)
    return y_pred_labels

def flag_sus_labels(npObject, labels_dict, labels, model, CON_THRESHOLD):
    predictions = model.predict(npObject.x, verbose = 0, batch_size = 32)
    y_pred = np.argmax(predictions, axis=-1)
    y_pred_labels = np.vectorize(labels_dict.get)(y_pred)
    unique, counts = np.unique(y_pred_labels, return_counts=True)

    fig, ax = plt.subplots(1,1,figsize=(6,3))
    ax.plot(y_pred,linestyle="",marker=".")
    plt.title('Predicted Activity Labels')
    ax.set_xlabel('Elapsed Time(seconds)')

    ax.set_yticks(np.arange(len(labels)))
    ax.set_yticklabels(labels)
    ax.grid()
    for i in range(0, len(predictions), 20):
        if sum([label_confidence(j)<CON_THRESHOLD for j in predictions[i:i+20]]) >0:
            plt.axvspan(i, i+20, color='gold', alpha=0.2)

    plt.tight_layout() # keeps labels from being cutoff when saving as pdf
    plt.show()
    return predictions

def labelDf(df, labels_dict, model, npObject):
    #copy the dataframe and initialize all predicted labels to Undefined
    labeledDf = df
    labeledDf['pred_labels'] = "Undefined"

    #reformat y_pred to hold the string values for labels
    predictions = model.predict(npObject.x, verbose = 0, batch_size = 32)
    y_pred = np.argmax(predictions, axis=-1)
    y_pred_labels = np.vectorize(labels_dict.get)(y_pred)
    unique, counts = np.unique(y_pred_labels, return_counts=True)

    #print out relevant information about labels and dictionary
    print("Final Predicted Label Counts: ")
    print (np.asarray((unique, counts)).T)
    #label the new dataframe

    for z in range(y_pred_labels.size):
        start = npObject.time[z][0]
        end = npObject.time[z][1]
        labeledDf.loc[start : end, ['pred_labels']] = y_pred_labels[z]

    return predictions

def confusion_mat(npObject, model, labelLoc):
    labels_dict = npObject.mapping

    predictions = model.model.predict(npObject.x, verbose = 0, batch_size = 32)
    y_pred = np.argmax(predictions, axis=-1)
    y_pred_labels = np.vectorize(labels_dict.get)(y_pred)
    unique, counts = np.unique(y_pred_labels, return_counts=True)

    test_acc , y_pred = model.run()
    y_test_act = np.argmax(npObject.y, axis=-1)
    class_rpt_for_log = classification_report(y_test_act, y_pred, target_names = unique)
    print(class_rpt_for_log)

    cm = confusion_matrix(y_test_act, y_pred)
    cm_df = pd.DataFrame(cm, index = unique, columns = unique)
    fig = plt.figure(figsize=(9,8))
    sns.heatmap(cm_df, annot=True, fmt='d', cmap= 'cubehelix_r')
    plt.title('1D CNN Accuracy:{0:.3f}'.format(accuracy_score(y_test_act, y_pred)))
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.tight_layout()
    plt.show()

info = widgets.Label('This button will add predicted labels to the dataset using the model you selected')
display(info)
def clicked(arg):
    print("Label Mapping: ", np_labeling.mapping)
    predictions = labelDf(dfA, np_labeling.mapping, testModel.model, np_labeling)
    global autoLabelPred
    autoLabelPred = predictions
    #flag_sus_labels(new_np, labels_to_dict(os.path.join(os.getcwd(), labelList)),os.path.join(os.getcwd(), 'testLabelList.csv'), testModel.model, 0.99)
    #confusion_mat(new_np ,testModel, os.path.join(os.getcwd(), labelList))

    labeled_file_path = os.path.join(working_dir, "assets", 'auto_label_df.csv')
    # Check if there's a datetime column in your DataFrame
    if 'datetime' in dfA.columns:
        dfA.to_csv(labeled_file_path, index=False)
    elif 'datetime' in dfA.index:
        dfA.reset_index(inplace=True)
        dfA.to_csv(labeled_file_path, index=False)
    print(f"Updated dataset saved to {labeled_file_path}")

    labeled_array_file_path = os.path.join(working_dir, "assets", "predictions.npy")
    np.save(labeled_array_file_path, predictions)
    print(f"NumPy predictions array saved to {labeled_array_file_path}")

button_download = widgets.Button(description = 'Add Labels')
button_download.on_click(clicked)
display(button_download)


# Review Auto Labels

### Specifications

In [ ]:
#@title Specification for Confidence Threshold
info1 = widgets.Label('If the predicted probability for a label is above this threshold, it will be considered a valid label. If it falls below this threshold, it will be flagged as suspect.')
display(info1)

info2 = widgets.Label('Enter value in range of 0-1.')
display(info2)

def on_button_clicked(b):
    conf_thresh = float(text_ui.value)
    print("Confidence Threshold is ", conf_thresh)
    # Path to your JSON configuration file
    config_file_path = 'assets/config.json'
    
    # Load the existing configuration
    try:
        with open(config_file_path, 'r') as file:
            config = json.load(file)
    except FileNotFoundError:
        config = {}  # If the file doesn't exist, start with an empty dictionary
    
    # Update the configuration with the new confidence threshold
    config['confidence_threshold'] = conf_thresh
    
    # Save the updated configuration back to the file
    with open(config_file_path, 'w') as file:
        json.dump(config, file, indent=4)
        
    return conf_thresh

description = widgets.Label("Enter Confidence Threshold: ")
text_ui = widgets.FloatText(value = 0.9)
button = widgets.Button(description="Submit")
button.on_click(on_button_clicked)

# Use HBox layout to place the text field and button side by side
hbox = widgets.HBox([description,text_ui])
display(hbox)
display(button)

# Third Dash App

In [ ]:
%run 3rd_app_dev.py

# Save Data

In [ ]:
# TO DO: New directory with necessary data everystep of the way with good labels for the files.